In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, RobustScaler, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [12]:
#  DATA LOADING

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

hospital_id_test = test["Hospital_Id"]
drop_cols = ["Hospital_Id", "Supplier_Name", "Hospital_Location"]
train = train.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)

target = "Transport_Cost"
train = train.dropna(subset=[target])

In [13]:
#  REMOVE OUTLIERS ABOVE 80th PERCENTILE

q80 = train[target].quantile(0.044)
train = train[train[target] >= q80]
print(f"Removed outliers above 80th percentile (>{q80:.2f}). Remaining samples: {len(train)}")

Removed outliers above 80th percentile (>-538.65). Remaining samples: 4780


In [14]:
#  FEATURE GROUPS

numeric_features_median = ['Equipment_Height', 'Equipment_Weight', 'Supplier_Reliability']
categorical_features_unknown = ['Equipment_Type', 'Transport_Method', 'Rural_Hospital']
no_missing_features_numerical = ['Equipment_Value', 'Base_Transport_Fee']
no_missing_features_categorical = ['Fragile_Equipment', 'Hospital_Info', 'CrossBorder_Shipping', 'Urgent_Shipping', 'Installation_Service']
date_features = ['Order_Placed_Date', 'Delivery_Date']

In [15]:
#  DATE FUNCTION

def compute_date_features(df_dates):
    df = df_dates.copy()
    df = df.apply(pd.to_datetime, format="%m/%d/%y", errors="coerce")
    order = df.iloc[:, 0]
    delivery = df.iloc[:, 1]
    delivery_days = (delivery - order).dt.days
    order_dow = order.dt.dayofweek.fillna(-1).astype(float)
    order_month = order.dt.month.fillna(0).astype(float)
    delivery_dow = delivery.dt.dayofweek.fillna(-1).astype(float)
    delivery_month = delivery.dt.month.fillna(0).astype(float)
    order_is_weekend = order_dow.isin([5, 6]).astype(float)
    delivery_is_weekend = delivery_dow.isin([5, 6]).astype(float)

    def cyc(x, period):
        xr = x.replace(-1, 0)
        rad = 2 * np.pi * xr / period
        return np.sin(rad), np.cos(rad)

    order_dow_sin, order_dow_cos = cyc(order_dow, 7)
    order_month_sin, order_month_cos = cyc(order_month, 12)

    features = pd.DataFrame({
        "delivery_days": delivery_days,
        "order_dow_sin": order_dow_sin,
        "order_dow_cos": order_dow_cos,
        "order_month_sin": order_month_sin,
        "order_month_cos": order_month_cos,
    })
    features.index = df_dates.index
    return features

In [16]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

no_missing_numerical_transformer = Pipeline([
    ('scaler', RobustScaler())
])

no_missing_categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

date_transformer = Pipeline([
    ('date_feat', FunctionTransformer(compute_date_features, validate=False)),
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('scaler', RobustScaler(with_centering=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features_median),
    ('cat_unknown', categorical_transformer, categorical_features_unknown),
    ('date', date_transformer, date_features),
    ('num_no_missing', no_missing_numerical_transformer, no_missing_features_numerical),
    ('cat_no_missing', no_missing_categorical_transformer, no_missing_features_categorical)
], remainder='drop')

In [17]:
# TRAIN-VALIDATION SPLIT

X = train.drop(columns=[target])
y = train[target].replace([np.inf, -np.inf], np.nan).fillna(0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine features and target for train set
train_processed = X_train.copy()
train_processed[target] = y_train

# Combine features and target for validation set
val_processed = X_val.copy()
val_processed[target] = y_val

# Save to CSV files
train_processed.to_csv('train_processed.csv', index=False)
val_processed.to_csv('val_processed.csv', index=False)

print(f"Train set saved as 'train_processed.csv' with {len(train_processed)} rows")
print(f"Validation set saved as 'val_processed.csv' with {len(val_processed)} rows")

Train set saved as 'train_processed.csv' with 3824 rows
Validation set saved as 'val_processed.csv' with 956 rows


In [18]:
#  MODEL DEFINITIONS

#  !!!! PLEASE READ THIS !!!! 

#  UNCOMMENT WHICHEVER MODEL YOU WANT TO TRY OUT BOTH IN THE MODELS AND THE PARAMETERS GRID BLOCK
#  YOU CAN USE MULTIPLE MODELS AT ONCE BY UNCOMMENTING THEM...
#  THIS WAS YOU CAN GET A PERFORMANCE COMPARISON USING THE VALIDATION DATA SET...

models = {
    # 'Linear Regression': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', TransformedTargetRegressor(
    #         regressor=LinearRegression(),
    #         transformer=PowerTransformer(method='yeo-johnson')
    #     ))
    # ]),
    
    # 'Lasso Regression': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', TransformedTargetRegressor(
    #         regressor=Lasso(max_iter=10000, random_state=42),
    #         transformer=PowerTransformer(method='yeo-johnson')
    #     ))
    # ]),
    
    # 'Elastic Net': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', TransformedTargetRegressor(
    #         regressor=ElasticNet(max_iter=10000, random_state=42),
    #         transformer=PowerTransformer(method='yeo-johnson')
    #     ))
    # ]),
    
    # 'K-Nearest Neighbors': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', KNeighborsRegressor())
    # ]),
    
    'Bayesian Ridge': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', TransformedTargetRegressor(
            regressor=BayesianRidge(),
            transformer=PowerTransformer(method='yeo-johnson')
        ))
    ]),
    
    # 'Decision Tree': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', DecisionTreeRegressor(random_state=42))
    # ]),
    
    # 'Random Forest': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', RandomForestRegressor(random_state=42, n_jobs=1))
    # ]),
    
    # 'Gradient Boosting': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', GradientBoostingRegressor(random_state=42))
    # ]),
    
    # 'AdaBoost': Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', AdaBoostRegressor(random_state=42))
    # ])
}


#  HYPERPARAMETER GRIDS

param_grids = {
    # 'Linear Regression': {},
    
    # 'Lasso Regression': {
    #     'regressor__regressor__alpha': [0.001, 0.01, 0.1, 1.0, 10.0]
    # },
    
    # 'Elastic Net': {
    #     'regressor__regressor__alpha': [0.001, 0.01, 0.1, 1.0],
    #     'regressor__regressor__l1_ratio': [0.2, 0.5, 0.8]
    # },
    
    # 'K-Nearest Neighbors': {
    #     'regressor__n_neighbors': [3, 5, 7, 9],
    #     'regressor__weights': ['uniform', 'distance']
    # },
    
    'Bayesian Ridge': {
        'regressor__regressor__alpha_1': [1e-6, 1e-5, 1e-4],
        'regressor__regressor__alpha_2': [1e-6, 1e-5, 1e-4]
    },
    
    # 'Decision Tree': {
    #     'regressor__max_depth': [3, 5, 7, 10, None],
    #     'regressor__min_samples_split': [2, 5, 10]
    # },
    
    # 'Random Forest': {
    #     'regressor__n_estimators': [50, 100, 200],
    #     'regressor__max_depth': [5, 10, None],
    #     'regressor__min_samples_split': [2, 5]
    # },
    
    # 'Gradient Boosting': {
    #     'regressor__n_estimators': [50, 100, 200],
    #     'regressor__learning_rate': [0.01, 0.1, 0.2],
    #     'regressor__max_depth': [3, 4, 5]
    # },
    
    # 'AdaBoost': {
    #     'regressor__n_estimators': [50, 100, 200],
    #     'regressor__learning_rate': [0.01, 0.1, 1.0]
    # }
}


In [19]:
#  MODEL TRAINING AND EVALUATION

print("="*70)
print("MODEL TRAINING AND PERFORMANCE ANALYSIS")
print("="*70)

results = {}
best_models = {}

for model_name in models.keys():
    print(f"\n Training {model_name}...")
    
    if param_grids[model_name]:
        # Use GridSearchCV for models with hyperparameters
        grid_search = GridSearchCV(
            models[model_name],
            param_grids[model_name],
            cv=5,
            scoring='r2',
            n_jobs=1,
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"   Best params: {best_params}")
    else:
        # Train directly for models without hyperparameters
        best_model = models[model_name]
        best_model.fit(X_train, y_train)
        best_params = "No hyperparameter tuning"
    
    # Store best model
    best_models[model_name] = best_model
    
    # Validation predictions
    y_pred_val = best_model.predict(X_val)
    
    # Train final model on full training data
    final_model = best_model
    final_model.fit(X, y)
    y_pred_train = final_model.predict(X)
    
    # Calculate metrics
    train_r2 = r2_score(y, y_pred_train)
    train_rmse = np.sqrt(mean_squared_error(y, y_pred_train))
    train_mae = mean_absolute_error(y, y_pred_train)
    
    val_r2 = r2_score(y_val, y_pred_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    val_mae = mean_absolute_error(y_val, y_pred_val)
    
    # Calculate MAPE (handle division by zero)
    try:
        train_mape = np.mean(np.abs((y - y_pred_train) / y)) * 100
        val_mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100
    except:
        train_mape = np.nan
        val_mape = np.nan
    
    # Store results
    results[model_name] = {
        'train_r2': train_r2,
        'train_rmse': train_rmse,
        'train_mae': train_mae,
        'train_mape': train_mape,
        'val_r2': val_r2,
        'val_rmse': val_rmse,
        'val_mae': val_mae,
        'val_mape': val_mape,
        'best_params': best_params
    }
    
    print(f"   {model_name} completed")
    print(f"   Training R²: {train_r2:.4f} | Validation R²: {val_r2:.4f}")
    print(f"   Training RMSE: {train_rmse:.2f} | Validation RMSE: {val_rmse:.2f}")


#  PERFORMANCE ANALYSIS

print("\n" + "="*70)
print("MODEL PERFORMANCE SUMMARY")
print("="*70)

# Create results dataframe
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('val_r2', ascending=False)

print("\n PERFORMANCE RANKING (by Validation R²):")
print("-" * 100)
print(f"{'Model':<25} {'Train R²':<10} {'Val R²':<10} {'Train RMSE':<12} {'Val RMSE':<12} {'Train MAE':<12} {'Val MAE':<12}")
print("-" * 100)

for model_name in results_df.index:
    r = results[model_name]
    print(f"{model_name:<25} {r['train_r2']:<10.4f} {r['val_r2']:<10.4f} {r['train_rmse']:<12.2f} {r['val_rmse']:<12.2f} {r['train_mae']:<12.2f} {r['val_mae']:<12.2f}")

# Find best model
best_model_name = results_df.index[0]
best_model = best_models[best_model_name]

print(f"\n MODEL: {best_model_name}")
print(f"   Validation R²: {results[best_model_name]['val_r2']:.4f}")
print(f"   Validation RMSE: {results[best_model_name]['val_rmse']:.2f}")

MODEL TRAINING AND PERFORMANCE ANALYSIS

 Training Bayesian Ridge...
   Best params: {'regressor__regressor__alpha_1': 0.0001, 'regressor__regressor__alpha_2': 1e-06}
   Bayesian Ridge completed
   Training R²: 0.0388 | Validation R²: 0.0144
   Training RMSE: 255769.83 | Validation RMSE: 297816.53

MODEL PERFORMANCE SUMMARY

 PERFORMANCE RANKING (by Validation R²):
----------------------------------------------------------------------------------------------------
Model                     Train R²   Val R²     Train RMSE   Val RMSE     Train MAE    Val MAE     
----------------------------------------------------------------------------------------------------
Bayesian Ridge            0.0388     0.0144     255769.83    297816.53    16723.77     14913.14    

 MODEL: Bayesian Ridge
   Validation R²: 0.0144
   Validation RMSE: 297816.53


In [20]:
#  DATA SUMMARY AND FINAL PREDICTIONS

print("\n" + "="*70)
print("DATA SUMMARY AND FINAL PREDICTIONS")
print("="*70)

print(f"\n DATA SUMMARY:")
print(f"   Training samples: {len(X)}")
print(f"   Validation samples: {len(X_val)}")
print(f"   Target variable range: {y.min():.2f} - {y.max():.2f}")
print(f"   Target variable mean: {y.mean():.2f}")
print(f"   Target variable std: {y.std():.2f}")

# Make predictions with best model
print(f"\n Making final predictions with best model: {best_model_name}")
y_pred_test = best_model.predict(test)

submission = pd.DataFrame({
    "Hospital_Id": hospital_id_test,
    "Transport_Cost": y_pred_test
})
submission.to_csv("submission.csv", index=False)

print(f" submission created successfully")
print(f"   Test predictions range: {y_pred_test.min():.2f} - {y_pred_test.max():.2f}")
print(f"   Test predictions mean: {y_pred_test.mean():.2f}")
print(f"   Test predictions std: {y_pred_test.std():.2f}")


DATA SUMMARY AND FINAL PREDICTIONS

 DATA SUMMARY:
   Training samples: 4780
   Validation samples: 956
   Target variable range: -538.58 - 11143428.25
   Target variable mean: 19341.05
   Target variable std: 260912.38

 Making final predictions with best model: Bayesian Ridge
 submission created successfully
   Test predictions range: -80.40 - 765318.70
   Test predictions mean: 5653.68
   Test predictions std: 41137.19
